In [1]:
# This is a quick bit of code that converts a pdf (when given the link) to a list of tokens


import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

nltk.download('stopwords')  
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/victor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#f = open('OSU-Out-with-Thumbnails.json')
json_file_path = "OKU-fact-sheets-out.json"
with open(json_file_path, 'r') as json_file:
    
    data = json.load(json_file)

In [5]:
links = [x['link'] for x in data]
print(links)

['https://extension.okstate.edu/fact-sheets/custom-beef-processing-expected-yields.html', 'https://extension.okstate.edu/fact-sheets/an-introduction-to-mental-health-challenges-for-rural-americans.html', 'https://extension.okstate.edu/fact-sheets/food-business-license-and-permit-costs-in-oklahoma-the-good-the-bad-and-the-mundane.html', 'https://extension.okstate.edu/fact-sheets/propagation-of-ornamental-plants-for-oklahoma.html', 'https://extension.okstate.edu/fact-sheets/cow-calf-production-record-software.html', 'https://extension.okstate.edu/fact-sheets/2021-2022-small-grains-variety-performance-tests.html', 'https://extension.okstate.edu/fact-sheets/shade-tree-borers.html', 'https://extension.okstate.edu/fact-sheets/machinery-ownership-versus-custom-harvest.html', 'https://extension.okstate.edu/fact-sheets/mobile-slaughter-units-regulatory-overview-for-oklahoma-non-poultry.html', 'https://extension.okstate.edu/fact-sheets/copper-sulfate-applications-for-filamentous-algae.html', 'ht

In [10]:
pdflinks=[]
for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("#main-content > div > div > div > div > div.share_button_container > a"):
        pdflinks.append(f"https://extension.okstate.edu/{link['href']}")
        print(pdflinks[links.index(i)])



https://extension.okstate.edu//fact-sheets/print-publications/fapc-food-and-agricultural-products-center/custom-beef-processing-expected-yields-fapc-245.pdf
https://extension.okstate.edu//fact-sheets/print-publications/t/an-introduction-to-the-mental-health-challenges-for-rural-americans-t-2150.pdf
https://extension.okstate.edu//fact-sheets/print-publications/fapc-food-and-agricultural-products-center/food-business-license-and-permit-costs-in-oklahoma-the-good-the-bad-and-the-mundane-fapc-185.pdf
https://extension.okstate.edu//fact-sheets/print-publications/e/propagation-of-ornamental-plants-for-oklahoma-e-917.pdf
https://extension.okstate.edu//fact-sheets/print-publications/cr/cow-calf-production-record-software-cr-3279.pdf
https://extension.okstate.edu//fact-sheets/print-publications/doc/cr-2143-2021-2022-small-grains-variety-performance-tests.pdf
https://extension.okstate.edu//fact-sheets/shade-tree-borers-pdf-only.pdf
https://extension.okstate.edu//fact-sheets/print-publications/ag

https://extension.okstate.edu//fact-sheets/print-publications/cr/protein-concentration-of-winter-wheat-varieties-in-oklahoma-2020-2021-cr-2135.pdf
https://extension.okstate.edu//fact-sheets/print-publications/aecl/overview-of-opportunities-for-beginning-farmers-and-ranchers-agec-2001.pdf
https://extension.okstate.edu//fact-sheets/print-publications/pss/2021-oklahoma-cotton-harvest-aid-guide-pss-2182.pdf
https://extension.okstate.edu//fact-sheets/print-publications/pss/impacts-of-winter-and-summer-crop-selection-on-the-performance-of-double-crop-systems-pss-3001.pdf
https://extension.okstate.edu//fact-sheets/print-publications/agec/liabilities-schedule-agec-792-1.pdf
https://extension.okstate.edu//fact-sheets/print-publications/agec/economically-optimal-planting-of-dual-purpose-wheat-agec-270.pdf
https://extension.okstate.edu//fact-sheets/print-publications/hla/algae-control-in-greenhouse-production-hla-6727.pdf
https://extension.okstate.edu//fact-sheets/print-publications/cr/should-i-b

https://extension.okstate.edu//fact-sheets/print-publications/agec/considering-water-quality-in-oklahoma-agec-1997.pdf
https://extension.okstate.edu//fact-sheets/print-publications/hla/cucumber-production-hla-6023.pdf
https://extension.okstate.edu//fact-sheets/print-publications/epp-entomology-and-plant-pathologhy/mites-in-small-grains-epp-7093.pdf
https://extension.okstate.edu//fact-sheets/print-publications/cr/oklahoma-cropland-rental-rates-2020-21cr-230.pdf
https://extension.okstate.edu//fact-sheets/print-publications/cr/oklahoma-pasture-rental-rates-2020-21-cr-216.pdf
https://extension.okstate.edu//fact-sheets/print-publications/hla/melon-production-hla-6237.pdf
https://extension.okstate.edu//fact-sheets/print-publications/nrem/using-drones-with-infrared-capabilities-to-monitor-fire-behavior-nrem-2907.pdf


KeyboardInterrupt: 

In [16]:
all_tokens=[]

def scrape_pdf(file, pdflink):
    wFile=file
    pdfreader = PyPDF2.PdfFileReader(BytesIO(wFile.read()))
    count=pdfreader.numPages
    words=list()
    words.append(pdflink)
    for j in range(count):
        pageObj = pdfreader.getPage(j)
        page1 = pageObj.extractText()
        punctuations = ['(',')',';',':','[',']',',','...','.']
        tokens = word_tokenize(page1)
        stop_words = stopwords.words('english')
        keywords = [word for word in tokens if not word in stop_words and not word in punctuations]
        for idx, token in enumerate(tokens):
            words.append(token)
        all_tokens.append(words)


        
def scrape_site(file):
    wFile=weasyprint.HTML(file).write_pdf()
    print(wFile)
    pdfreader = PyPDF2.PdfFileReader(BytesIO(wFile.read()))
    count=pdfreader.numPages
    words=list()
    for j in range(count):
        pageObj = pdfreader.getPage(j)
        page1 = pageObj.extractText()
        punctuations = ['(',')',';',':','[',']',',','...','.']
        tokens = word_tokenize(page1)
        stop_words = stopwords.words('english')
        keywords = [word for word in tokens if not word in stop_words and not word in punctuations]
        for idx, token in enumerate(tokens):
            words.append(token)
        all_tokens.append(words)

for i in range(len(pdflinks)):
    print(pdflinks[i])
    try:
        wFile=urllib.request.urlopen(pdflinks[i])
        scrape_pdf(wFile, pdflinks[i])
    except:
        pass
            

        

https://extension.okstate.edu//fact-sheets/print-publications/fapc-food-and-agricultural-products-center/custom-beef-processing-expected-yields-fapc-245.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.


https://extension.okstate.edu//fact-sheets/print-publications/t/an-introduction-to-the-mental-health-challenges-for-rural-americans-t-2150.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


https://extension.okstate.edu//fact-sheets/print-publications/fapc-food-and-agricultural-products-center/food-business-license-and-permit-costs-in-oklahoma-the-good-the-bad-and-the-mundane-fapc-185.pdf
https://extension.okstate.edu//fact-sheets/print-publications/e/propagation-of-ornamental-plants-for-oklahoma-e-917.pdf


...

Xref table not zero-indexed. ID numbers for objects will be corrected.


https://extension.okstate.edu//fact-sheets/print-publications/cr/oklahoma-pasture-rental-rates-2020-21-cr-216.pdf
https://extension.okstate.edu//fact-sheets/print-publications/hla/melon-production-hla-6237.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.


https://extension.okstate.edu//fact-sheets/print-publications/nrem/using-drones-with-infrared-capabilities-to-monitor-fire-behavior-nrem-2907.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.


In [17]:
print(all_tokens[0])

['https://extension.okstate.edu//fact-sheets/print-publications/fapc-food-and-agricultural-products-center/custom-beef-processing-expected-yields-fapc-245.pdf', 'Division', 'of', 'Agricultural', 'Sciences', 'and', 'Natural', 'Resources', '•', 'Oklahoma', 'State', 'UniversityFAPC-245', 'Oklahoma', 'Cooperative', 'Extension', 'Fact', 'Sheets', 'are', 'also', 'available', 'on', 'our', 'website', 'at', ':', 'extension.okstate.eduOklahoma', 'Cooperative', 'Extension', 'Service', 'September', '2022', 'Joel', 'Jackson', 'Meat', 'Pilot', 'Manager', ',', 'FAPC', 'Recent', 'meat', 'shortages', 'have', 'resulted', 'in', 'an', 'increase', 'of', 'first-time', 'custom', 'slaughter', 'customers', '.', 'There', 'are', 'significant', 'differences', 'in', 'finished', 'product', 'weight', 'based', 'off', 'multiple', 'factors', '.', 'The', 'following', 'information', 'will', 'give', 'customers', 'an', 'idea', 'of', 'what', 'they', 'can', 'expect', 'when', 'picking', 'up', 'their', 'custom', 'processed', '